<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/ngram_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language Modeling

### Goal: compute a probabilty distribution over all possible sentences:


### $$p(W) = p(w_1, w_2, ..., w_T)$$

### This unsupervised learning problem can be framed as a sequence of supervised learning problems:

### $$p(W) = p(w_1) * p(w_2|w_1) * ... * p(w_T|w_1, ..., w_{T-1})$$

### If we have K sentences, where the j-th sentence has T_j words for all j frmo 1 to K, then we want to max:

### $$log p(W) = \sum_{j = 1}^K \sum_{i=1}^{T_j} log p(w_i | w_{<i})$$




# N-gram language model

### Goal: estimate the n-gram probabilities using counts of sequences of n consecutive words

### Given a sequence of words $w$, we want to compute

###  $$P(w_i|w_{i−1}, w_{i−2}, …, w_{i−n+1})$$

### Where $w_i$ is the i-th word of the sequence.

### $$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) = \frac{p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w_i)}{\sum_{w \in V} p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w)}$$

### Key Idea: We can estimate the probabilities using counts of n-grams in our dataset 


## N-gram Probabilities

## $$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) \approx \frac{c(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w_i)}{\sum_{w \in V} c(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w)}$$


## Bigram Probabilities

## $$p(w_i | w_{i-1}) = \frac{c(w_{i-1}, w_i)}{\sum_{w_i} c(w_{i-1}, w_i)} $$


### 1. Data Processing
### 2. Ngram Counts and Probabilities
### 3. Trie Implementation
### 4. Smoothing
### 5. Perplexity
### 6. Sentence Score
### 7. Sentence Generation

In [1]:
# !python -m spacy download en_core_web_sm

In [2]:
import os
import sys
sys.path.append('utils/')
from utils import ngram_utils as ngram_utils
import utils.global_variables as gl
import torch
import random
from utils.ngram_utils import NgramLM

In [3]:
torch.manual_seed(1)

### Load Data from .txt Files

In [4]:
# Read data from .txt files and create lists of reviews

train_data = []
# create a list of all the reviews 
with open('../data/amazon_train.txt', 'r') as f:
    train_data = [review for review in f.read().split('\n') if review]
    
valid_data = []
# create a list of all the reviews 
with open('../data/amazon_valid.txt', 'r') as f:
    valid_data = [review for review in f.read().split('\n') if review]
    

In [5]:
# type(train_data), len(train_data), \
# type(train_data[0]), len(train_data[0]), \
# type(train_data[0][0]), len(train_data[0][0])

In [6]:
train_data[0], train_data[0][0], len(train_data)


("this is a great tutu and at a really great price . it doesn ' t look cheap at all . i ' m so glad i looked on amazon and found such an affordable tutu that isn ' t made poorly . a + + ",
 't',
 22288)

### Process the Data

In [7]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = ngram_utils.tokenize_dataset(train_data)
valid_data_tokenized, all_tokens_valid = ngram_utils.tokenize_dataset(valid_data)


Let's look at the tokenized data!

In [8]:
# # Number of All Tokens
# len(all_tokens_train), all_tokens_train[0], \
len(train_data_tokenized), train_data_tokenized[0]

(107790,
 ['this',
  'is',
  'a',
  'great',
  'tutu',
  'and',
  'at',
  'a',
  'really',
  'great',
  'price',
  '.'])

In [9]:
train_ngram_lm = NgramLM(train_data_tokenized, all_tokens_train, n=3, smoothing=None)
valid_ngram_lm = NgramLM(valid_data_tokenized, all_tokens_valid, n=3, smoothing=None)

In [10]:
train_ngram_lm.trie_ngram['./<eos>/<eos>']

96175

In [11]:
train_ngram_lm.n, train_ngram_lm.frac_vocab

(3, 0.9)

In [12]:
valid_ngram_lm.id2token[0:10]

['<unk>', '<sos>', '<eos>', '.', 'the', 'i', ',', 'and', 'a', 'it']

In [13]:
valid_ngram_lm.token2id['<unk>'], valid_ngram_lm.token2id['<sos>'], valid_ngram_lm.token2id['the']

(0, 1, 4)

In [14]:
valid_ngram_lm.vocab_ngram[:10], valid_ngram_lm.count_ngram[:10]

((('.', '<eos>', '<eos>'),
  ('<sos>', '<sos>', 'i'),
  ('<sos>', '<sos>', 'the'),
  ('<sos>', '<sos>', 'it'),
  ('!', '<eos>', '<eos>'),
  ('<sos>', '<sos>', 'this'),
  ('it', "'", 's'),
  ('.', '.', '.'),
  ('.', '.', '<eos>'),
  ('<sos>', '<sos>', 'they')),
 (13625, 3635, 1425, 1100, 1049, 762, 687, 655, 580, 569))

In [15]:
valid_ngram_lm.vocab_bigram[:10], valid_ngram_lm.count_bigram[:10]

((('.', '<eos>'),
  ('<sos>', 'i'),
  ('<sos>', 'the'),
  ("'", 't'),
  ("'", 's'),
  ('.', '.'),
  ('<sos>', 'it'),
  ('!', '<eos>'),
  (',', 'and'),
  (',', 'but')),
 (13625, 3635, 1425, 1261, 1249, 1238, 1100, 1049, 900, 838))

In [16]:
valid_ngram_lm.vocab_unigram[:10], valid_ngram_lm.count_unigram[:10]

((('.',),
  ('the',),
  ('i',),
  (',',),
  ('and',),
  ('a',),
  ('it',),
  ('to',),
  ("'",),
  ('is',)),
 (14883, 9408, 8000, 7525, 6226, 5774, 5085, 4550, 3816, 3695))

In [17]:
valid_ngram_lm.vocab_prev_ngram[:10], valid_ngram_lm.count_prev_ngram[:10]

((('.', '<eos>'),
  ('<sos>', 'i'),
  ('<sos>', 'the'),
  ("'", 't'),
  ("'", 's'),
  ('.', '.'),
  ('<sos>', 'it'),
  ('!', '<eos>'),
  (',', 'and'),
  (',', 'but')),
 (13625, 3635, 1425, 1261, 1249, 1238, 1100, 1049, 900, 838))

In [18]:
valid_ngram_lm.id2token_ngram[:10]

[('.', '<eos>', '<eos>'),
 ('<sos>', '<sos>', 'i'),
 ('<sos>', '<sos>', 'the'),
 ('<sos>', '<sos>', 'it'),
 ('!', '<eos>', '<eos>'),
 ('<sos>', '<sos>', 'this'),
 ('it', "'", 's'),
 ('.', '.', '.'),
 ('.', '.', '<eos>'),
 ('<sos>', '<sos>', 'they')]

In [19]:
valid_ngram_lm.token2id_ngram[('.', '<eos>', '<eos>')], valid_ngram_lm.token2id_ngram[('.', '.', '<eos>')]

(0, 8)

#### Build the Vocabulary 


In [20]:
# Build a vocabulary using all the tokens found in train data (90% of most common ones)
print('Word vocabulary size: {} words'.format(len(train_ngram_lm.token2id)))        

Word vocabulary size: 20806 words


### CORPUS ANALYSIS (Train + Valid Data)

#### Number of Tokens in the Corpus Data


In [21]:
print("Number of All Tokens ", len(all_tokens_train))

Number of All Tokens  1623446


#### Number of Sentences in the Train Data


In [22]:
print("Number of Sentences ", len(train_ngram_lm.raw_data))

Number of Sentences  107790


## N-grams

In [23]:
n = 3 # trigrams

### Function for padding the sentences with special markers sentence beginning and end, i.e. $<bos>$ and $<eos>$

In [24]:
train_padded = train_ngram_lm.padded_data
train_ngram = train_ngram_lm.ngram_data
vocab_ngram = train_ngram_lm.vocab_ngram
count_ngram = train_ngram_lm.count_ngram 

In [25]:
train_padded[0]

['<sos>',
 '<sos>',
 'this',
 'is',
 'a',
 'great',
 'tutu',
 'and',
 'at',
 'a',
 'really',
 'great',
 'price',
 '.',
 '<eos>',
 '<eos>']

### Function for finding all N-grams

In [26]:
train_ngram[0]

[('<sos>', '<sos>', 'this'),
 ('<sos>', 'this', 'is'),
 ('this', 'is', 'a'),
 ('is', 'a', 'great'),
 ('a', 'great', 'tutu'),
 ('great', 'tutu', 'and'),
 ('tutu', 'and', 'at'),
 ('and', 'at', 'a'),
 ('at', 'a', 'really'),
 ('a', 'really', 'great'),
 ('really', 'great', 'price'),
 ('great', 'price', '.'),
 ('price', '.', '<eos>'),
 ('.', '<eos>', '<eos>')]

In [27]:
vocab_ngram[0]

('.', '<eos>', '<eos>')

In [28]:
count_ngram[0]

96175

In [29]:
trie_ngram = train_ngram_lm.trie_ngram
# trie_ngram
# trie_prev_ngram = train_ngram_lm.trie_prev_ngram

In [30]:
trie_ngram['./<eos>/<eos>']

96175

In [31]:
id2token = train_ngram_lm.id2token
token2id = train_ngram_lm.token2id

In [32]:
id2token_ngram = train_ngram_lm.id2token_ngram
token2id_ngram = train_ngram_lm.token2id_ngram

In [33]:
random_token_id = random.randint(0, len(id2token_ngram) - 1)
random_token = id2token_ngram[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token_ngram[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id_ngram[random_token]))

Token id 574532 ; token ('58', 'mm', 'which')
Token ('58', 'mm', 'which'); token id 574532


### Ngram Count & Probability

In [34]:
# TODO: print the words for which the pd is nonzero !!! -- more intuitive than a list of numbers

In [35]:
vocab_ngram[:10], count_ngram[:10]

((('.', '<eos>', '<eos>'),
  ('<sos>', '<sos>', 'i'),
  ('<sos>', '<sos>', 'the'),
  ('!', '<eos>', '<eos>'),
  ('<sos>', '<sos>', 'they'),
  ('<sos>', '<sos>', 'it'),
  ('.', '.', '.'),
  ('<sos>', '<sos>', 'this'),
  ('<sos>', '<sos>', 'these'),
  ('.', '.', '<eos>')),
 (96175, 26986, 9197, 8152, 6376, 5373, 4693, 4189, 3941, 3876))

In [36]:
c = train_ngram_lm.get_ngram_count(('an', 'older', 'coat'))
p = train_ngram_lm.get_ngram_prob(('an', 'older', 'coat'))

p1 = train_ngram_lm.get_ngram_prob(('an', 'older', 'pc'))
p2 = train_ngram_lm.get_ngram_prob(('an', 'older', 'lady'))
p3 = train_ngram_lm.get_ngram_prob(('an', 'older', 'watch'))

pd = train_ngram_lm.get_prob_distr_ngram(('an', 'older'))

c, p, p1, p2, p3, sum(pd)#, pd

(1,
 0.041666666666666664,
 0.041666666666666664,
 0.041666666666666664,
 0.0,
 0.9999999999999999)

In [37]:
c = train_ngram_lm.get_ngram_count(('really', 'great', 'price'))
p = train_ngram_lm.get_ngram_prob(('really', 'great', 'price'))
pd = train_ngram_lm.get_prob_distr_ngram(('really', 'great'))

c, p, sum(pd)#, pd 

(3, 0.061224489795918366, 0.9999999999999997)

In [38]:
c = train_ngram_lm.get_ngram_count(('really', 'great'))

c

0

In [39]:
c = train_ngram_lm.get_ngram_count(('.', '<eos>', '<eos>'))
p = train_ngram_lm.get_ngram_prob(('.', '<eos>', '<eos>'))
pd = train_ngram_lm.get_prob_distr_ngram(('.', '<eos>'))

c, p, sum(pd)#, pd

(96175, 1.0, 1.0)

In [40]:
c = train_ngram_lm.get_ngram_count(('.', '<sos>', '<sos>'))

c

0

In [41]:
c = train_ngram_lm.get_ngram_count(('i', 'like', 'pandas'))
p = train_ngram_lm.get_ngram_count(('i', 'like', 'pandas'))
pd = train_ngram_lm.get_prob_distr_ngram(('i', 'like'))

c, p, sum(pd)#, pd

(0, 0, 0.9999999999999994)

In [42]:
c = train_ngram_lm.get_ngram_count(('is', 'a', 'great'))
p = train_ngram_lm.get_ngram_prob(('is', 'a', 'great'))
pd = train_ngram_lm.get_prob_distr_ngram(('is', 'a'))

c, p, sum(pd)#, pd

(266, 0.09819121447028424, 0.9999999999999787)

In [43]:
c = train_ngram_lm.get_ngram_count(('send', 'it', 'back'))
p = train_ngram_lm.get_ngram_prob(('send', 'it', 'back'))
pd = train_ngram_lm.get_prob_distr_ngram(('send', 'it', 'back'))

c, p, sum(pd)#, pd

(28, 0.9032258064516129, 1.0000000000000657)

In [44]:
c = train_ngram_lm.get_ngram_count(('i', 'like', 'these', 'pictures'))
p = train_ngram_lm.get_ngram_prob(('i', 'like', 'these', 'pictures'))
pd = train_ngram_lm.get_prob_distr_ngram(('i', 'like', 'these'))

c, p, sum(pd)#, pd

(0, 0, 1.0000000000000657)

## Add-One Smoothing

## $$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) \approx \frac{1 + c(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w_i)}{\mid V\mid + \sum_{w \in V} c(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w)}$$


In [45]:
p = train_ngram_lm.get_ngram_prob_add_one_smoothing(('.', '<sos>', '<sos>'))
p

4.806305873305777e-05

In [46]:
p = train_ngram_lm.get_ngram_prob_add_one_smoothing(('i', 'like', 'pandas'))
p

4.504910352283989e-05

In [47]:
p = train_ngram_lm.get_ngram_prob_add_one_smoothing(('i', 'like', 'this'))
p

0.00432471393819263

In [48]:
p = train_ngram_lm.get_ngram_prob_add_one_smoothing(('really', 'great', 'price'))
p

0.0001918005274514505

In [49]:
p = train_ngram_lm.get_ngram_prob_add_one_smoothing(('send', 'it', 'back'))
p

0.0013917550511110045

In [50]:
p = train_ngram_lm.get_ngram_prob_add_one_smoothing(('.', '<eos>', '<eos>'))
p

0.8221506056539096

## Additive Smoothing

## $$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) \approx \frac{\delta + c(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w_i)}{\delta\mid V\mid + \sum_{w \in V} c(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w)}$$


In [51]:
p = train_ngram_lm.get_ngram_prob_additive_smoothing(('.', '<sos>', '<sos>'), delta = 0.5)
p

4.806305873305777e-05

In [52]:
p = train_ngram_lm.get_ngram_prob_additive_smoothing(('i', 'like', 'pandas'), delta = 0.5)
p

4.23908435777872e-05

In [53]:
p = train_ngram_lm.get_ngram_prob_additive_smoothing(('i', 'like', 'this'), delta = 0.5)
p

0.008096651123357355

In [54]:
p = train_ngram_lm.get_ngram_prob_additive_smoothing(('really', 'great', 'price'), delta = 0.5)
p

0.00033486414083429007

In [55]:
p = train_ngram_lm.get_ngram_prob_additive_smoothing(('send', 'it', 'back'), delta = 0.5)
p

0.0027314548591144336

In [56]:
p = train_ngram_lm.get_ngram_prob_additive_smoothing(('.', '<eos>', '<eos>'), delta = 0.5)
p

0.9023954287001069

### Changing the Parameter $\delta$

In [57]:
# small delta --> closer to no smoothing  (1.0)
p = train_ngram_lm.get_ngram_prob_additive_smoothing(('.', '<eos>', '<eos>'), delta = 0.1)
p

0.9788256343658784

In [58]:
# arge delta --> closer to add-one smoothing (0.58)
p = train_ngram_lm.get_ngram_prob_additive_smoothing(('.', '<eos>', '<eos>'), delta = 0.9)
p

0.8370371208455323

## Linear Interpolation Smoothing (Jelinek-Mercer)

### $$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) \approx \alpha_n P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) + (1 - \alpha_n) P(w|w_{i−n+2}, ..., w_{i−2}, w_{i−1})$$


In [59]:
p = train_ngram_lm.get_ngram_prob_interpolation_smoothing(('.', '<sos>', '<sos>'), alpha = 0.8)
p

0.0

In [60]:
p = train_ngram_lm.get_ngram_prob_interpolation_smoothing(('i', 'like', 'pandas'), alpha = 0.8)
p

0.0

In [61]:
p = train_ngram_lm.get_ngram_prob_interpolation_smoothing(('i', 'like', 'this'), alpha = 0.8)
p

0.0545977011494253

In [62]:
p = train_ngram_lm.get_ngram_prob_interpolation_smoothing(('really', 'great', 'price'), alpha = 0.8)
p

0.0489795918367347

In [63]:
p = train_ngram_lm.get_ngram_prob_interpolation_smoothing(('send', 'it', 'back'), alpha = 0.8)
p

0.7225806451612904

### Changing the Parameter $\alpha$

In [64]:
# small delta --> closer to no smoothing  (1.0)
p = train_ngram_lm.get_ngram_prob_interpolation_smoothing(('.', '<eos>', '<eos>'), alpha = 0.8)
p

0.8

In [65]:
# small delta --> closer to no smoothing  (1.0)
p = train_ngram_lm.get_ngram_prob_interpolation_smoothing(('.', '<eos>', '<eos>'), alpha = 0.5)
p

0.5

In [66]:
# small delta --> closer to no smoothing  (1.0)
p = train_ngram_lm.get_ngram_prob_interpolation_smoothing(('.', '<eos>', '<eos>'), alpha = 0.2)
p

0.2

## Linear Interpolation with Absolute Discounting

### $$p_{bi}(w|v) = max ({ \frac{N(v, w) - b_{bi}}{N(v)}, 0)  + b_{bi} \frac{V - N_0(v, \cdot)}{N(v)} p_{uni}(w) \large}$$

### $$p_{uni}(w) = max ({ \frac{N(w) - b_{uni}}{N}, 0)  + b_{uni} \frac{V - N_0(\cdot)}{N} \frac{1}{V}}$$

### $$b_{bi} = \frac{N_1(\cdot, \cdot)}{N_1(\cdot, \cdot) + 2*N_2(\cdot, \cdot)}$$

### $$b_{uni} = \frac{N_1(\cdot)}{N_1(\cdot) + 2*N_2(\cdot)}$$


### $$N_r(\cdot) = \sum_{w: N(w) = r} 1$$

### $$N_r(\cdot, \cdot) = \sum_{v, w: N(v, w) = r} 1$$

### $$N_r(v, \cdot) = \sum_{w: N(v, w) = r} 1$$

### V is the number of words in the vocabulary

### $N_r(\cdot, \cdot)$ and $N_r(\cdot)$  are the count-counts for bigrams and unigrams respectively $


### Remember to check that probabilities sum up to one:
### $$\sum_w p_{bi}(w|v) = \sum_w p_{uni}(w) = 1$$



In [67]:
# y = "m"
# x = "'"

# z = train_ngram_lm.get_p_bi(y, x)
# z

In [68]:
train_ngram[:3]

[[('<sos>', '<sos>', 'this'),
  ('<sos>', 'this', 'is'),
  ('this', 'is', 'a'),
  ('is', 'a', 'great'),
  ('a', 'great', 'tutu'),
  ('great', 'tutu', 'and'),
  ('tutu', 'and', 'at'),
  ('and', 'at', 'a'),
  ('at', 'a', 'really'),
  ('a', 'really', 'great'),
  ('really', 'great', 'price'),
  ('great', 'price', '.'),
  ('price', '.', '<eos>'),
  ('.', '<eos>', '<eos>')],
 [('<sos>', '<sos>', 'it'),
  ('<sos>', 'it', 'doesn'),
  ('it', 'doesn', "'"),
  ('doesn', "'", 't'),
  ("'", 't', 'look'),
  ('t', 'look', 'cheap'),
  ('look', 'cheap', 'at'),
  ('cheap', 'at', 'all'),
  ('at', 'all', '.'),
  ('all', '.', '<eos>'),
  ('.', '<eos>', '<eos>')],
 [('<sos>', '<sos>', 'i'),
  ('<sos>', 'i', "'"),
  ('i', "'", 'm'),
  ("'", 'm', 'so'),
  ('m', 'so', 'glad'),
  ('so', 'glad', 'i'),
  ('glad', 'i', 'looked'),
  ('i', 'looked', 'on'),
  ('looked', 'on', 'amazon'),
  ('on', 'amazon', 'and'),
  ('amazon', 'and', 'found'),
  ('and', 'found', 'such'),
  ('found', 'such', 'an'),
  ('such', 'an', 'af

## Kneser-Ney Smoothing (best to use in practice!) http://smithamilli.com/blog/kneser-ney/

### Bigram LM
###  $$p(s) = \prod_{i = 1} ^ {N + 1} p(w_i | w_{i-1})$$

## Likelihood of a Sentence

### Bigram LM: $$ p(i \; love \; this \; light) = p(i|\cdot) \; p(love|i)\;  p(this|love)\;  p(light|this) \\
\approx \frac{c(i, \cdot)}{\sum_w c(\cdot, \; w)} \; \frac{c(love, i)}{\sum_wc(i, \; w)}\;  \frac{c(this, love)}{\sum_wc(love, \;w)}\;  \frac{c(light, this)}{\sum_wc(this, \;w)}$$ 

### Trigram LM: $$ p(i \; love \; this  \;light) = p(i|\cdot, \cdot) \; p(love|\cdot, i) \; p(this|i, love)\;  p(light|love, this)$$ 



### Score Sentences

In [ ]:
n = 3
sentence = [['this', 'is', 'a', 'great', 'tutu']]
print(sentence)
ps = train_ngram_lm.get_prob_sentence(sentence)
ss =  train_ngram_lm.get_score_sentence(sentence)
ps, ss

[['this', 'is', 'a', 'great', 'tutu']]


(0.0, 5.126374389946876e+77)

In [ ]:
n = 3
sentence = [['this', 'is', 'a', 'great', 'tutu', 'and', 'at', 'a', 'really', 'great', 'price', '.']]
print(sentence)
ps = train_ngram_lm.get_prob_sentence(sentence)
ss = train_ngram_lm.get_score_sentence(sentence)
ps, ss

[['this', 'is', 'a', 'great', 'tutu', 'and', 'at', 'a', 'really', 'great', 'price', '.']]


(4.0043653926145575e-13, 6.706528173768987)

## Sentence Generation

#### No Context

In [ ]:
num_tokens = 5
generated_sentence = train_ngram_lm.generate_sentence(num_tokens)
generated_sentence


i
i think
i think they
i think they can
i think they can be


'i think they can be'

In [ ]:
num_tokens = 10
generated_sentence = train_ngram_lm.generate_sentence(num_tokens, context=('i', 'like', 'the'))
generated_sentence


ones
ones here
ones here ,
ones here , let
ones here , let '
ones here , let ' s
ones here , let ' s well
ones here , let ' s well constructed
ones here , let ' s well constructed and
ones here , let ' s well constructed and fit


"ones here , let ' s well constructed and fit"

In [ ]:
num_tokens = 20
generated_sentence = train_ngram_lm.generate_sentence(num_tokens)
generated_sentence


on
on this
on this product
on this product and
on this product and the
on this product and the second
on this product and the second bag
on this product and the second bag in
on this product and the second bag in a
on this product and the second bag in a store
on this product and the second bag in a store .
on this product and the second bag in a store . <eos>


'on this product and the second bag in a store . <eos>'

#### With Context

In [ ]:
num_tokens = 5
generated_sentence = train_ngram_lm.generate_sentence(num_tokens, context=('i', 'like', 'the'))
generated_sentence


fact
fact that
fact that now
fact that now .
fact that now . <eos>


'fact that now . <eos>'

In [ ]:
num_tokens = 10
generated_sentence = train_ngram_lm.generate_sentence(num_tokens, context=('i', 'like', 'the'))
generated_sentence


moss
moss color
moss color so
moss color so i
moss color so i went
moss color so i went back
moss color so i went back and
moss color so i went back and helps
moss color so i went back and helps you
moss color so i went back and helps you !


'moss color so i went back and helps you !'

In [ ]:
num_tokens = 20
generated_sentence = train_ngram_lm.generate_sentence(num_tokens, context=('i', 'like', 'the'))
generated_sentence


color
color and
color and style
color and style .


In [ ]:
num_tokens = 10
generated_sentence = train_ngram_lm.generate_sentence(num_tokens, context=('the', 'worst'))
generated_sentence


In [ ]:
num_tokens = 5
generated_sentence = train_ngram_lm.generate_sentence(num_tokens, context=('the', 'best'))
generated_sentence


In [ ]:
num_tokens = 5
generated_sentence = train_ngram_lm.generate_sentence(num_tokens, context=('not', 'what'))
generated_sentence


In [ ]:
num_tokens = 5
generated_sentence = train_ngram_lm.generate_sentence(num_tokens, context=('i', 'will'))
generated_sentence


## Log-Likelihood (n-gram)
## $$LL = \sum_{j=1}^{K} \sum_{i=1}^{T_j + 1} log p_{bi}(w_{j, i} | w_{j, n - i + 1}, \cdot, w_{j, i - 2}, w_{j, i - 1})$$

## Perplexity
## $$PP = exp(-\frac{LL}{\sum_j(T_j + 1)})$$

In [ ]:
ppl_train = train_ngram_lm.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid = train_ngram_lm.get_perplexity(valid_data_tokenized, subsample=10)


In [ ]:
ppl_valid, ppl_train

### Interpolation Smoothing - varying N

In [ ]:
# Interpolation Smoothing, N = 2
train_ngram_lm_interp2 = NgramLM(train_data_tokenized, all_tokens_train, n=2, smoothing='interpolation')
valid_ngram_lm_interp2 = NgramLM(valid_data_tokenized, all_tokens_valid, n=2, smoothing='interpolation')

ppl_train_no_interp2 = train_ngram_lm_interp2.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_interp2 = train_ngram_lm_interp2.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_interp2, ppl_train_no_interp2


In [ ]:
# Interpolation Smoothing, N = 3
train_ngram_lm_interp3 = NgramLM(train_data_tokenized, all_tokens_train, n=3, smoothing='interpolation')
valid_ngram_lm_interp3 = NgramLM(valid_data_tokenized, all_tokens_valid, n=3, smoothing='interpolation')

ppl_train_no_interp3 = train_ngram_lm_interp3.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_interp3 = train_ngram_lm_interp3.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_interp3, ppl_train_no_interp3


In [ ]:
# Interpolation Smoothing, N = 5
train_ngram_lm_interp5 = NgramLM(train_data_tokenized, all_tokens_train, n=5, smoothing='interpolation')
valid_ngram_lm_interp5 = NgramLM(valid_data_tokenized, all_tokens_valid, n=5, smoothing='interpolation')

ppl_train_no_interp5 = train_ngram_lm_interp5.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_interp5 = train_ngram_lm_interp5.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_interp5, ppl_train_no_interp5


In [ ]:
# Interpolation Smoothing, N = 7
train_ngram_lm_interp7 = NgramLM(train_data_tokenized, all_tokens_train, n=7, smoothing='interpolation')
valid_ngram_lm_interp7 = NgramLM(valid_data_tokenized, all_tokens_valid, n=7, smoothing='interpolation')

ppl_train_no_interp7 = train_ngram_lm_interp7.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_interp7 = train_ngram_lm_interp7.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_interp7, ppl_train_no_interp7


In [ ]:
# Interpolation Smoothing, N = 10
train_ngram_lm_interp10 = NgramLM(train_data_tokenized, all_tokens_train, n=10, smoothing='interpolation')
valid_ngram_lm_interp10 = NgramLM(valid_data_tokenized, all_tokens_valid, n=10, smoothing='interpolation')

ppl_train_no_interp10 = train_ngram_lm_interp10.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_interp10 = train_ngram_lm_interp10.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_interp10, ppl_train_no_interp10


### Let's Compare Different Smoothing Techniques

In [ ]:
# No Smoothing
train_ngram_lm_no_smoothing = NgramLM(train_data_tokenized, all_tokens_train, n=7)
valid_ngram_lm_no_smoothing = NgramLM(valid_data_tokenized, all_tokens_valid, n=7)

ppl_train_no_smoothing = train_ngram_lm_no_smoothing.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_smoothing = train_ngram_lm_no_smoothing.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_smoothing, ppl_train_no_smoothing


In [ ]:
# Additive Smoothing
train_ngram_lm_additive = NgramLM(train_data_tokenized, all_tokens_train, n=7, smoothing='additive', delta=0.5)
valid_ngram_lm_additive = NgramLM(valid_data_tokenized, all_tokens_valid, n=7, smoothing='additive', delta=0.5)

ppl_train_no_additive = train_ngram_lm_additive.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_additive = train_ngram_lm_additive.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_additive, ppl_train_no_additive


In [ ]:
# Additive Smoothing
train_ngram_lm_additive_d2 = NgramLM(train_data_tokenized, all_tokens_train, n=7, smoothing='additive', delta=0.2)
valid_ngram_lm_additive_d2 = NgramLM(valid_data_tokenized, all_tokens_valid, n=7, smoothing='additive', delta=0.2)

ppl_train_no_additive_d2 = train_ngram_lm_additive_d2.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_additive_d2 = train_ngram_lm_additive_d2.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_additive_d2, ppl_train_no_additive_d2


In [ ]:
# Additive Smoothing
train_ngram_lm_additive_d8 = NgramLM(train_data_tokenized, all_tokens_train, n=7, smoothing='additive', delta=0.8)
valid_ngram_lm_additive_d8 = NgramLM(valid_data_tokenized, all_tokens_valid, n=7, smoothing='additive', delta=0.8)

ppl_train_no_additive_d8 = train_ngram_lm_additive_d8.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_additive_d8 = train_ngram_lm_additive_d8.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_additive_d8, ppl_train_no_additive_d8


In [ ]:
# Additive Smoothing
train_ngram_lm_add1 = NgramLM(train_data_tokenized, all_tokens_train, n=7, smoothing='add-one')
valid_ngram_lm_add1 = NgramLM(valid_data_tokenized, all_tokens_valid, n=7, smoothing='add-one')

ppl_train_no_add1 = train_ngram_lm_add1.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_add1 = train_ngram_lm_add1.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_add1, ppl_train_no_add1


In [ ]:
# Interpolation Smoothing
train_ngram_lm_interp_a2 = NgramLM(train_data_tokenized, all_tokens_train, n=7, smoothing='interpolation', alpha=0.2)
valid_ngram_lm_interp_a2 = NgramLM(valid_data_tokenized, all_tokens_valid, n=7, smoothing='interpolation', alpha=0.2)

ppl_train_no_interp_a2 = train_ngram_lm_interp_a2.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_interp_a2 = train_ngram_lm_interp_a2.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_interp_a2, ppl_train_no_interp_a2


In [ ]:
# Interpolation Smoothing
train_ngram_lm_interp_a8 = NgramLM(train_data_tokenized, all_tokens_train, n=7, smoothing='interpolation', alpha=0.8)
valid_ngram_lm_interp_a8 = NgramLM(valid_data_tokenized, all_tokens_valid, n=7, smoothing='interpolation', alpha=0.8)

ppl_train_no_interp_a8 = train_ngram_lm_interp_a8.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_interp_a8 = train_ngram_lm_interp_a8.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_interp_a8, ppl_train_no_interp_a8


In [ ]:
# Interpolation Smoothing
train_ngram_lm_interp_a5 = NgramLM(train_data_tokenized, all_tokens_train, n=7, smoothing='interpolation', alpha=0.5)
valid_ngram_lm_interp_a5 = NgramLM(valid_data_tokenized, all_tokens_valid, n=7, smoothing='interpolation', alpha=0.5)

ppl_train_no_interp_a5 = train_ngram_lm_interp_a5.get_perplexity(train_data_tokenized, subsample=10)
ppl_valid_no_interp_a5 = train_ngram_lm_interp_a5.get_perplexity(valid_data_tokenized, subsample=10)

ppl_valid_no_interp_a5, ppl_train_no_interp_a5


In [ ]:
# # Discounted Interpolation Smoothing
# train_ngram_lm_discount = NgramLM(train_data_tokenized, all_tokens_train, n=3, smoothing='discounting')
# valid_ngram_lm_discount = NgramLM(valid_data_tokenized, all_tokens_valid, n=3, smoothing='discounting')

# ppl_train_no_discount = train_ngram_lm_discount.get_perplexity(train_data_tokenized)
# ppl_valid_no_discount = train_ngram_lm_discount.get_perplexity(valid_data_tokenized)

# ppl_valid_no_discount, ppl_train_no_discount


### Additive Smoothing - varying N

### Sentence Probabilities

In [ ]:
sentence = [['this', 'is', 'a', 'great', 'tutu']]
print(sentence)
ps = train_ngram_lm.get_prob_sentence(sentence)
ss = train_ngram_lm.get_score_sentence(sentence)
ps, ss

In [ ]:
sentence = [['this', 'is', 'a', 'great', 'tutu', 'and', 'at', 'a', 'really', 'great', 'price', '.']]
print(sentence)
ps = train_ngram_lm_interp3.get_prob_sentence(sentence)
ss = train_ngram_lm_interp3.get_score_sentence(sentence)
ps, ss

In [ ]:
sentence = [['this', 'is', 'a', 'great', 'tutu', 'and', 'at', 'a', 'really', 'great', 'price', '.']]
print(sentence)
ps = train_ngram_lm_interp5.get_prob_sentence(sentence)
ss = train_ngram_lm_interp5.get_score_sentence(sentence)
ps, ss

In [ ]:
sentence = [['this', 'is', 'a', 'great', 'tutu', 'and', 'at', 'a', 'really', 'great', 'price', '.']]
print(sentence)
ps = train_ngram_lm_interp7.get_prob_sentence(sentence)
ss = train_ngram_lm_interp7.get_score_sentence(sentence)
ps, ss

In [ ]:
sentence = [['this', 'is', 'a', 'great', 'tutu']]
print(sentence)
ps = train_ngram_lm_interp10.get_prob_sentence(sentence)
ss = train_ngram_lm_interp10.get_score_sentence(sentence)
ps, ss

In [ ]:
sentence = [['this', 'is', 'a', 'great', 'tutu', 'and', 'at', 'a', 'really', 'great', 'price', '.']]
print(sentence)
ps = train_ngram_lm_additive.get_prob_sentence(sentence)
ss = train_ngram_lm_additive.get_score_sentence(sentence)
ps, ss

In [ ]:
sentence = [['i', 'like', 'pandas']]
print(sentence)
ps = train_ngram_lm_additive.get_prob_sentence(sentence)
ss = train_ngram_lm_additive.get_score_sentence(sentence)
ps, ss

In [ ]:
sentence = [['i really like this watch']]
print(sentence)
ps = train_ngram_lm_additive.get_prob_sentence(sentence)
ss = train_ngram_lm_additive.get_score_sentence(sentence)
ps, ss

In [ ]:
sentence = [['my wife really likes the color of this dress']]
print(sentence)
ps = train_ngram_lm_additive.get_prob_sentence(sentence)
ss = train_ngram_lm_additive.get_score_sentence(sentence)
ps, ss

### Sentence Generation

In [ ]:
num_tokens = 10
generated_sentence = train_ngram_lm_interp3.generate_sentence(num_tokens)
generated_sentence


In [ ]:
num_tokens = 10
generated_sentence = train_ngram_lm_interp5.generate_sentence(num_tokens)
generated_sentence

In [ ]:
num_tokens = 10
generated_sentence = train_ngram_lm_interp7.generate_sentence(num_tokens)
generated_sentence


In [ ]:
num_tokens = 10
generated_sentence = train_ngram_lm_interp10.generate_sentence(num_tokens)
generated_sentence


In [ ]:
num_tokens = 20
generated_sentence = train_ngram_lm_interp10.generate_sentence(num_tokens)
generated_sentence


In [ ]:
num_tokens = 10
generated_sentence = train_ngram_lm_additive.generate_sentence(num_tokens)
generated_sentence


In [ ]:
num_tokens = 10
generated_sentence = train_ngram_lm_no_smoothing.generate_sentence(num_tokens)
generated_sentence
